<a href="https://colab.research.google.com/github/jonnells/cmpe255-project-grp/blob/main/Yelp_On_The_Go.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yelp On the Go!

In [1]:
import os
import json
import time

## Import Data

In [2]:
yelp_dataset_path = 'yelp_dataset/'

In [3]:
user_data = []
start = time.time()
with open(yelp_dataset_path + 'yelp_academic_dataset_user.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_user.json')
    for line in fin:
        user_data.append(json.loads(line))
end = time.time()
duration_without_dr = end-start
print("Time taken to load user data: %d seconds" %duration_without_dr)

Reading yelp_academic_dataset_user.json
Time taken to load user data: 23 seconds


In [4]:
business_data = []
start = time.time()
with open(yelp_dataset_path + 'yelp_academic_dataset_business.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_business.json')
    for line in fin:
        business_data.append(json.loads(line))

end = time.time()
duration_without_dr = end-start
print("Time taken to load business data: %d seconds" %duration_without_dr)

Reading yelp_academic_dataset_business.json
Time taken to load business data: 2 seconds


## Data Investigation

In [5]:
print(str(len(user_data)) + ' user records loaded')
#print(str(len(review_data)) + ' review records loaded')
print(str(len(business_data)) + ' business records loaded')

2189457 user records loaded
160585 business records loaded


In [6]:
print(user_data[0].keys())

dict_keys(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny', 'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot', 'compliment_more', 'compliment_profile', 'compliment_cute', 'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool', 'compliment_funny', 'compliment_writer', 'compliment_photos'])


In [7]:
#print(review_data[0].keys())

In [8]:
print(business_data[0].keys())

dict_keys(['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours'])


## Data Pre-processing

Remove users that do not have enough reviews (10)

In [9]:
user_data = [user for user in user_data if user.get('review_count') >= 15]

In [10]:
print(str(len(user_data)) + ' useful user records')

573316 useful user records


Only keep restaurant related businesses

In [11]:
print(business_data[0])

{'business_id': '6iYb2HFDywm3zjuRg0shjw', 'name': 'Oskar Blues Taproom', 'address': '921 Pearl St', 'city': 'Boulder', 'state': 'CO', 'postal_code': '80302', 'latitude': 40.0175444, 'longitude': -105.2833481, 'stars': 4.0, 'review_count': 86, 'is_open': 1, 'attributes': {'RestaurantsTableService': 'True', 'WiFi': "u'free'", 'BikeParking': 'True', 'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}", 'BusinessAcceptsCreditCards': 'True', 'RestaurantsReservations': 'False', 'WheelchairAccessible': 'True', 'Caters': 'True', 'OutdoorSeating': 'True', 'RestaurantsGoodForGroups': 'True', 'HappyHour': 'True', 'BusinessAcceptsBitcoin': 'False', 'RestaurantsPriceRange2': '2', 'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': False, 'intimate': False, 'trendy': False, 'upscale': False, 'classy': False, 'casual': True}", 'HasTV': 'True', 'Alcohol': "'beer_and_wine'", 'GoodForMeal': "{'dessert': False, 'latenight': Fa

In [12]:
restaurant_category = 'Restaurants'
restaurant_data = []
for business in business_data:
    try:
        if restaurant_category in business.get('categories',):
            restaurant_data.append(business)
    except:
        pass
        

#restaurant_data = [business for business in business_data if restaurant_category in business.get('categories', default=None)]

In [13]:
print(str(len(restaurant_data)) + ' restaurant records')

50763 restaurant records


Grab imporant attributes

In [14]:
restaurant_ids = [restaurant.get('business_id') for restaurant in restaurant_data]

In [15]:
restaurant_stars = [restaurant.get('stars') for restaurant in restaurant_data]

In [16]:
user_ids = [user.get('user_id') for user in user_data]

In [17]:
states = [restaurant.get('state') for restaurant in restaurant_data]
tempset = set(states)
states = sorted(list(tempset))
print(states)
restaurant_len = [1, 7506, 865, 7710, 6140, 1, 1, 10550, 1, 1, 4377, 7391, 5444, 1, 773, 1]
restaurants_by_state = {states[i]: restaurant_len[i] for i in range(len(states))}
print('Number of restaurants by state\n',restaurants_by_state)

['ABE', 'BC', 'CO', 'FL', 'GA', 'KS', 'KY', 'MA', 'MN', 'NH', 'OH', 'OR', 'TX', 'VA', 'WA', 'WY']
Number of restaurants by state
 {'ABE': 1, 'BC': 7506, 'CO': 865, 'FL': 7710, 'GA': 6140, 'KS': 1, 'KY': 1, 'MA': 10550, 'MN': 1, 'NH': 1, 'OH': 4377, 'OR': 7391, 'TX': 5444, 'VA': 1, 'WA': 773, 'WY': 1}


In [18]:
#ABE_restaurants = [restaurant for restaurant in restaurant_data if 'ABE' in restaurant.get('state')]
#BC_restaurants = [restaurant for restaurant in restaurant_data if 'BC' in restaurant.get('state')]
CO_restaurants = [restaurant for restaurant in restaurant_data if 'CO' in restaurant.get('state')]
#FL_restaurants = [restaurant for restaurant in restaurant_data if 'FL' in restaurant.get('state')]
#GA_restaurants = [restaurant for restaurant in restaurant_data if 'GA' in restaurant.get('state')]
#KS_restaurants = [restaurant for restaurant in restaurant_data if 'KS' in restaurant.get('state')]
#KY_restaurants = [restaurant for restaurant in restaurant_data if 'KY' in restaurant.get('state')]
MA_restaurants = [restaurant for restaurant in restaurant_data if 'MA' in restaurant.get('state')]
#MN_restaurants = [restaurant for restaurant in restaurant_data if 'MN' in restaurant.get('state')]
#NH_restaurants = [restaurant for restaurant in restaurant_data if 'NH' in restaurant.get('state')]
#OH_restaurants = [restaurant for restaurant in restaurant_data if 'OH' in restaurant.get('state')]
#OR_restaurants = [restaurant for restaurant in restaurant_data if 'OR' in restaurant.get('state')]
#TX_restaurants = [restaurant for restaurant in restaurant_data if 'TX' in restaurant.get('state')]
#VA_restaurants = [restaurant for restaurant in restaurant_data if 'VA' in restaurant.get('state')]
#WA_restaurants = [restaurant for restaurant in restaurant_data if 'WA' in restaurant.get('state')]
#WY_restaurants = [restaurant for restaurant in restaurant_data if 'WY' in restaurant.get('state')]








#restaurant_len = [len(ABE_restaurants),len(BC_restaurants),len(CO_restaurants),len(FL_restaurants),len(GA_restaurants),len(KS_restaurants),len(KY_restaurants),len(MA_restaurants),len(MN_restaurants),len(NH_restaurants),len(OH_restaurants),len(OR_restaurants),len(TX_restaurants),len(VA_restaurants),len(WA_restaurants),len(WY_restaurants)]

In [19]:
with open('MA_restaurants.json', 'a') as outfile:
    json.dump(MA_restaurants, outfile, indent=1)

In [20]:
review_data = []
MA_restaurants_id = set([restaurant.get('business_id') for restaurant in MA_restaurants])
valid_user_id = set([user.get('user_id') for user in user_data])
start = time.time()
print(time.asctime())
with open(yelp_dataset_path + 'yelp_academic_dataset_review.json', encoding="utf8") as fin:
    print('Reading yelp_academic_dataset_review.json')
    for line in fin:
        review = json.loads(line)
        if review.get('business_id') in MA_restaurants_id:
            if review.get('user_id') in valid_user_id:
                review_data.append(review)

end = time.time()
duration_without_dr = end-start
print("Time taken to load review data: %d seconds" %duration_without_dr)

Sat May 22 13:07:42 2021
Reading yelp_academic_dataset_review.json
Looking at 100000  reviews at 0 seconds
Looking at 200000  reviews at 1 seconds
Looking at 300000  reviews at 2 seconds
Looking at 400000  reviews at 2 seconds
Looking at 500000  reviews at 3 seconds
Looking at 600000  reviews at 4 seconds
Looking at 700000  reviews at 4 seconds
Looking at 800000  reviews at 5 seconds
Looking at 900000  reviews at 6 seconds
Looking at 1000000  reviews at 6 seconds
Looking at 1100000  reviews at 7 seconds
Looking at 1200000  reviews at 7 seconds
Looking at 1300000  reviews at 8 seconds
Looking at 1400000  reviews at 9 seconds
Looking at 1500000  reviews at 9 seconds
Looking at 1600000  reviews at 10 seconds
Looking at 1700000  reviews at 11 seconds
Looking at 1800000  reviews at 11 seconds
Looking at 1900000  reviews at 12 seconds
Looking at 2000000  reviews at 13 seconds
Looking at 2100000  reviews at 13 seconds
Looking at 2200000  reviews at 14 seconds
Looking at 2300000  reviews at 15

In [21]:
with open('MA_reviews.json', 'a') as outfile:
    json.dump(review_data, outfile, indent=1)

In [22]:
print(review_data[0].keys())

dict_keys(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date'])


In [23]:
MA_users = []
MA_user_id = set([review.get('user_id') for review in review_data])
for user in user_data:
    if user.get('user_id') in MA_user_id:
        MA_users.append(user)

with open('MA_users.json', 'a') as outfile:
    json.dump(MA_users, outfile, indent=1)


